In [1]:
%matplotlib inline

In [2]:
import collections
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp_sparse
import tables
import seaborn as sns
from collections import defaultdict
from __future__ import division
import tqdm

/Users/nranu/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def Read10x(filepath):   
    with tables.open_file(filepath, 'r') as f:
        try:
            group = f.get_node(f.root)
        except tables.NoSuchNodeError:
            print "That genome does not exist in this file."
        gene_ids = getattr(group, 'gene_ids').read()
        gene_names = getattr(group, 'gene_names').read()
        gene = getattr(group, 'gene').read()
        umi_corrected_reads=getattr(group, 'umi_corrected_reads').read()
        nonconf_mapped_reads=getattr(group, 'nonconf_mapped_reads').read()
        conf_mapped_uniq_read_pos=getattr(group, 'conf_mapped_uniq_read_pos').read()
        unmapped_reads=getattr(group, 'unmapped_reads').read()
        barcodes = getattr(group, 'barcode').read()
        reads = getattr(group, 'reads').read()
        umi = getattr(group, 'umi').read()

        TABLE=pd.DataFrame()
        TABLE['bc']=barcodes
        TABLE['umi']=umi
        #TABLE['bcumi']=zip(barcodes,umi)
        TABLE['gene']=gene
        TABLE['unique']=[1]*len(TABLE)
        TABLE['map_logical']=conf_mapped_uniq_read_pos>0
        TABLE['read_counts']=reads+nonconf_mapped_reads+unmapped_reads
        return TABLE
def setenv(newDict):
    DNA={}
    DNA['A']='00'
    DNA['C']='01'
    DNA['G']='10'
    DNA['T']='11'

    BC_DNA2={}
    for i in newDict.iterkeys(): 
        string=newDict[i]
        for j in DNA.iterkeys():
            string=string.replace(j,DNA[j])
        BC_DNA2[i]=int(string,2)
    return BC_DNA2
np.random.seed(0)

GeneBCMatrix = collections.namedtuple('GeneBCMatrix', ['gene_ids', 'gene_names', 'barcodes', 'matrix'])
def get_matrix_from_h5(filename, genome):
    with tables.open_file(filename, 'r') as f:
        try:
            dsets = {}
            for node in f.walk_nodes('/' + genome, 'Array'):
                dsets[node.name] = node.read()
            matrix = sp_sparse.csc_matrix((dsets['data'], dsets['indices'], dsets['indptr']), shape=dsets['shape'])
            return GeneBCMatrix(dsets['genes'], dsets['gene_names'], dsets['barcodes'], matrix)
        except tables.NoSuchNodeError:
            raise Exception("Genome %s does not exist in this file." % genome)
        except KeyError:
            raise Exception("File is missing one or more required datasets.")
def get_gene(BC,df_1):
    Total=df_1.gene[df_1.bc==BC].value_counts().to_frame()
    Total.columns=['Counts']
    genes1=np.zeros(32739)
    for index,rows in Total.iterrows():
        genes1[index]=rows['Counts']
    return genes1[:-1]

def get_sample(Tot,reads,temp_sample1_1,barcode):    
    if reads[barcode]<Tot[Tot.bc==barcode].Total_reads.sum():
        a=reads[barcode]
        down=True
    elif reads[barcode]>Tot[Tot.bc==barcode].Total_reads.sum():
         down=False
    if item==17:#PH_23,4
        down=False
    temp_3=temp_sample1_1[temp_sample1_1.Total_reads>2]
    

    if down==True:
        temp_sample3=temp_3.sample(a)
    elif down==False:
        temp_sample3=temp_3.copy()


    temp_sample3=temp_sample3.drop_duplicates()
    return temp_sample3
def Expanded_dataframe(tot3):
    #converts UMI table to reads table for sampling
    tot3=tot3.reset_index()
    temp_tot=defaultdict()
    counter=0
    for item in tot3.index:
        counter1=0
        number=tot3.loc[item]['Total_reads']
        while counter1<number:
            temp_tot[counter]=tot3.loc[item].values
            counter+=1
            counter1+=1
    temp_tot2=pd.DataFrame.from_dict(temp_tot)
    temp_tot2=temp_tot2.T
    temp_tot2.columns=tot3.columns
    return temp_tot2

In [ ]:
newDict = {}
#insert path to barcode text file
with open('/../enriched_barcodes.txt', 'r') as f:
    for line in f:
        splitLine = line.split(',')
        newDict[int(splitLine[0])] = splitLine[1][:-1]
BC=setenv(newDict)

In [ ]:
#insert path to non enriched deep sequencing molecule info H5 file from 10X pipeline
c_path='/../molecule_info.h5'
control=Read10x(c_path)

In [ ]:
#CD_19, insert gene expression profile path
#filtered_matrix_h5='/.../filtered_gene_bc_matrices_h5.h5'

#HLA_DR, insert gene expression profile path
filtered_matrix_h5='/../filtered_gene_bc_matrices_h5.h5'

genome = "hg19"

#load expression profile
%time gene_bc_matrix = get_matrix_from_h5(filtered_matrix_h5, genome)

In [ ]:
gnames=gene_bc_matrix.gene_names
cnames=gene_bc_matrix.barcodes
DGE=pd.DataFrame(gene_bc_matrix.matrix.toarray())
DGE.index=gnames
DGE.columns=cnames
cumi=DGE.sum()

In [ ]:
#CD-19

#list_a=[1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]


#HLA-DR

list_a=[4,5,6,7,8,9]
list_b=[12,13,14,15,16,17,30,31,32,33]
list_c=[34,35,36,37,38,39,40,41,42,43]
list_d=[44,45,46,47,48,49,50,51,52,53]
list_e=[54,55,56,58,59,60,61,62,63,64,65,68]


list_f=[4,5,6,7,8,9,12,13,14,15,16,17,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,48,49,50,51,52,53,54,55,56,58,59,60,61,62,63,64,65,68]

In [ ]:
#convert all barcodes to 2-bit encoded
barcodes=defaultdict(str)
for index, item in enumerate(cnames):
        barcodes[index]=item[:-2]
Tot_BC=setenv(barcodes)

#reads per barcode for all barcodes
reads=defaultdict(int)
for k,v in Tot_BC.iteritems():
    reads[v]=control[control.bc==v]['read_counts'].sum()

In [ ]:
samples=[1,2,3] #replicate samples for one enrichment pool
t_path=#path to data
sample_path1='NR_'+str(samples[0])+'/'
sample_path2='NR_'+str(samples[1])+'/'
sample_path3='NR_'+str(samples[2])+'/'
file_path='molecule_info.h5'
NR1_=Read10x(t_path+sample_path1+file_path)
NR2_=Read10x(t_path+sample_path2+file_path)
NR3_=Read10x(t_path+sample_path3+file_path)
NR1_['Sample']=1
NR2_['Sample']=2
NR3_['Sample']=3
NR_tot=pd.concat([NR1_,NR2_,NR3_])

In [ ]:
def get_expanded_dataframe(NR_tot,list_a,BC,reads):    
    t=NR_tot.copy()
    t['bcumig']=zip(t.bc,t.umi,t.gene)
    BCUMI_group=t.groupby('bcumig').sum()
    BCUMI_group=pd.DataFrame(BCUMI_group[['unique','read_counts']])
    BCUMI_group.columns=['Number','Total_reads']
    BCUMI_group['BCUMIG']=BCUMI_group.index
    Tot=BCUMI_group[['Number','Total_reads','BCUMIG']]

    Tot['bc']=[x for x,y,z in Tot.BCUMIG]
    Tot['umi']=[y for x,y,z in Tot.BCUMIG]
    Tot['gene']=[z for x,y,z in Tot.BCUMIG]
    

    barcode=BC[item] 
    temp_sample1_1=Expanded_dataframe(Tot[Tot.bc==barcode])
    return Tot,temp_sample1_1

In [ ]:
bias=defaultdict()

In [ ]:
for item in tqdm.tqdm_notebook(list_d,desc='0th loop'): 
    barcode=BC[item]
    
    boot=defaultdict()
    Tot,temp_1=get_expanded_dataframe(NR_tot,item,BC,reads)
    for i in tqdm.tqdm_notebook(range(10),desc='1st',leave=False):
        temp=get_sample(Tot,reads,temp_1,barcode)
        con_sample=control[control.bc==barcode]


        control3=pd.DataFrame(control[['bc','umi','gene']])
        control3['Enrich']='control'
        sample3=pd.DataFrame(temp[['bc','umi','gene']])
        sample3['Enrich']='enrich'
        check2=pd.concat([control3[control3.bc!=barcode],sample3])
        check3=check2[check2.gene!=32738]
        check3['umig']=zip(check3.umi,check3.gene)
        check3['val']=1
        
        BCUMI_group2=check3.groupby('umig').sum()
        BCUMI_group2=pd.DataFrame(BCUMI_group2['val'])
        BCUMI_group2.columns=['Number']
        check4=check3.copy()
        check4.index=check4['umig']
        check4=check4.join(BCUMI_group2)
        check5=check4[check4.Enrich=='enrich']
        boot[i]=check5.Number.value_counts()
    bias[item]=pd.DataFrame.from_dict(boot,orient='index').mean(axis=0)

In [ ]:
bias_df=pd.DataFrame.from_dict(bias,orient='index')